In [55]:
import numpy as np
import cv2
import itertools
import time
from numba import njit,jit,prange,vectorize,int32,float64,int64, cuda
import random
import tensorflow as tf
import glob
quesize = 20
k = 0

In [47]:
input_size = 256*3

@njit
def preprocessing(a):
    px_train = a.reshape(input_size)
    px_train = np.asarray(px_train).astype(np.int32)

    return px_train  

In [48]:
class WiSARDrp:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos


In [49]:
input_size = 256*3
no_of_rand_pix_selec = 2**(2)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 240*250

In [50]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(60000, 3072, 2)


In [13]:
@njit(parallel = True)
def train_with_bleeching(d,pos,x_train, y_train):
    B = 50    
    images = x_train
    lable = y_train    
    image = images
    num = lable
    all_ram_of_selected_discriminator = d[num]
    t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
    for i in range((int)(nodes)):
        part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
        ratina_for_one_ram = t_ratina[i]
                
            #threshold = 0               
        n = []                                                                
        for ix in range(len(ratina_for_one_ram)):
            pix = ratina_for_one_ram[ix]
            if image[(pix-1)]>=1:
                n.append(1)
                    #threshold = threshold + 1     
            else:
                n.append(0)
                
        num = 0
        for i in range(no_of_rand_pix_selec):
            num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
        address_of_that_ram = (int)(num)
        for key in range(ram_address_count):
            index = part[key]
                #if threshold >= 1:          
            if index[0] == address_of_that_ram and index[1] < B:
                index[1] += 1
                #x = 0
            else:
                if index[1] != 0 and index[1] != 0:
                    index[1] -= 1
                    #x = 0

                        
@njit(parallel = True)
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    b=1
        
    #for i in range(len(images)):
    image = images
    actual_lable = lable
            
    ix = actual_lable       
    #for ix in range(dis_number):
                
    t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
    sum_of_ram_output = 0
    dis = d[ix]
                
    for i in range(int(nodes)):
        part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
        ratina_for_one_ram = t_ratina[i]
                    
        n = []                                                                
        for pix in ratina_for_one_ram:
            if image[(pix-1)]>=1:
                n.append(1)
            else:
                n.append(0)
                    
        num = 0
        for i in range(no_of_rand_pix_selec):
            num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
        address_of_that_ram = (int)(num)
                
        for key in range(len(part)):
            prt = part[key]
            if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                sum_of_ram_output += 1
            
            
    if sum_of_ram_output/nodes >= 0.75:
        right += 1
    else:
        wrong += 1
        
        
    return right,wrong                
                
                

@njit
def IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel):
    stabilityqueue2 = stabilityqueue1
    c = np.zeros((quesize,3),dtype=np.int32) 
    b = np.zeros((4,3),dtype=np.int32)
    for queueIndex in range(quesize):
        queuevalue = np.reshape(stabilityqueue2[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0] == pixel[0][0] and  queuevalue[1][0] == pixel[1][0] and  queuevalue[2][0] == pixel[2][0] :
            b = np.roll(stabilityqueue2[i,j,:queueIndex+1,:],3) 
            c[:queueIndex+1] = b
            c[queueIndex+2:] = stabilityqueue2[i,j,queueIndex+2:,:]
            stabilityqueue2[i,j,:,:] = c
            return stabilityqueue2,True
    #print(stabilityqueue2.shape)
    return stabilityqueue2,False    
    
    
    
#@njit
#@jit
def bewis(frame,w,ID,stabilityqueue,out):
    w1 = w
    ID1 = ID
    stabilityqueue1 = stabilityqueue
    out1 = out
    #print(w1)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            #print(i)
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            a = np.zeros((256,3))
            a[0:r,0] = 1  
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            #print(a)
            front = np.reshape(stabilityqueue1[i,j,0,:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            pixel_no = i*10+j
            #print(111)
            if front[0][0] == pixel[0][0] and  front[1][0] == pixel[1][0] and  front[2][0] == pixel[2][0] :
                w1[i,j] += ID1[i,j]
                #print(w1[i,j])
                ID1[i,j] += 1
                #print(w1[i,j])
                #continue
            #print(333)    
            #stabilityqueue1,found = IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel)
            #print(222)
            if front[0][0] != pixel[0][0] or  front[1][0] != pixel[1][0] or  front[2][0] != pixel[2][0] :
                stabilityqueue1,found = IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel)
                if found:
                    ID1[i,j] = 1
                #print(1)
                else:
                    stabilityqueue1[i,j,:,:] = np.roll(stabilityqueue1[i,j,:,:],1)
                #print(stabilityqueue1[i,j,0,:].shape)
                    stabilityqueue1[i,j,0,:]= np.reshape(pixel,(3,))
                    w1[i,j] = 0
                #print(2)
                    ID1[i,j] = 1
            
            #print(w1[i,j])       #here i am getting 0
            if w1[i,j] > k:
                #print(w1[i,j])
                #print(pixel_no)
                #print(1)
                pixel_train = preprocessing(a)
                #print(pixel_train.shape)
                train_with_bleeching(d,acc_pos,pixel_train,pixel_no)
                #print(pixel.shape)
                #print(pixel.dtype)
                #print(i,j)
                #print(a)
                #print(a.shape)
            #print(w1[i,j])
            #reconstruct colors from neurons and write it on BG image
            pixel_test = preprocessing(a)
            #print(pixel_no)
            #right = 0
            #wrong = 1
            right,wrong = test_with_bleaching(d,acc_pos,pixel_test,pixel_no)
            #print(right)
            if right:
                
                out1[i,j] = 0.
                #print(out1)
            if wrong:
                #print(5)
                out1[i,j] = 1.
                #print(0)

    #print(w1)    
    return w1,ID1,stabilityqueue1,out1



def getframe():
    stabilityqueue = np.zeros((125,150,quesize,3),dtype=np.int32)   #240,256  #480,640 for bird video, 720,1280 for E_IP1231_Day_1
    w = np.zeros((125,150),dtype=np.int32)
    ID = np.zeros((125,150),dtype=np.int32)
    cap = cv2.VideoCapture('day.avi')
    out = np.ones((125,150),dtype=np.float64)
    n = 0
    #frame_number = 0
    while(True):
        #frame = cv2.imread('Stonehenge.jpg')
        ret, frame = cap.read()
        #Display the resulting frame
        #print(frame.shape)
        #if n <= 3:
            #ret, frame1 = cap.read()
            #cv2.imshow('frame1',frame1)
            #frame = frame
            #print(frame1.shape)
            #n += 1
        width = 150
        height = 125
        dim = (width, height)
 
        # resize image
        #resized = frame
        resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) 
        cv2.imshow('frame',resized)
        #print(frame.shape)
        #print(resized.shape)
        w1, ID1, stabilityqueue1,out1 = bewis(resized,w,ID,stabilityqueue,out)
        w = w1
        ID = ID1
        stabilityqueue = stabilityqueue1
        out = out1
        #print(w)
        #print(out)
        kernel = np.ones((4,4),np.uint8)
        erosion = cv2.erode(out,kernel,iterations = 1)
        dilation = cv2.dilate(erosion,kernel,iterations = 1)
        cv2.imshow('out',dilation)
        #frame_number += 1
        #print(frame_number)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


getframe()
'''
    print(frame.shape)
    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):
        print (p)
        #r,g,b = frame[p[0],p[1],:]
        #a = np.zeros((256,3))
'''        

'\n    print(frame.shape)\n    for p in itertools.product(range(frame.shape[0]),range(frame.shape[1])):\n        print (p)\n        #r,g,b = frame[p[0],p[1],:]\n        #a = np.zeros((256,3))\n'

In [1]:
import time
from numba import njit,prange,vectorize,int32,float64,int64,jit
import numpy as np
import random
import tensorflow as tf

In [2]:
def preprocessing(tx_train, ty_train, tx_test, ty_test):
        
    py_train = ty_train.flatten()
    py_test = ty_test.flatten()
    
    px_test = tx_test.reshape(10000,input_size)
    px_train = tx_train.reshape(60000,input_size)
    
    px_train = np.asarray(px_train).astype(np.int32)
    py_train = np.asarray(py_train).astype(np.int32)
    px_test = np.asarray(px_test).astype(np.int32)
    py_test = np.asarray(py_test).astype(np.int32)

    return px_train, py_train, px_test, py_test   


In [3]:
class WiSARDrp:                                                     
    
    def __init__(self,input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number):
        self.input_size = input_size
        self.no_of_rand_pix_selec = no_of_rand_pix_selec
        self.nodes = nodes
        self.ram_address_count = ram_address_count
        self.dis_number = dis_number 


    def discriminator(self):
        discriminator = []
        accumulated_pos = []
        my_list = list(range(0,self.input_size))
        for i in range(self.dis_number):  #10
            ram = []
            random.shuffle(my_list)
            for j in range((int)((self.nodes))): #98    
                total_pos = []            
                positions = []
                positions = my_list[j*no_of_rand_pix_selec:j*no_of_rand_pix_selec+no_of_rand_pix_selec]
                accumulated_pos.append(positions)
                total_pos = np.vstack(positions)
                #print(total_pos)
                table = []
                dictionary = {}
                
                max = len("{0:b}".format(2**len(total_pos))) - 1
                for i in range(2**len(total_pos)):
                    x = (('0' * max) + "{0:b}".format(i))
                    x = x[len(x)-max:]
                    dictionary[x] = 0
                table.append(dictionary)
                #print(table)
                ram.append(table)
        
            di = []
            for j in range(len(ram)):
                for i in range(len(ram[j])):
                    for key, value in ram[j][i].items():
                        temp = [key,value]
                        di.append(temp)
                        
            discriminator.append(di)
    
        discriminator = np.asarray(discriminator).astype(np.int32)
        accumulated_pos = np.asarray(accumulated_pos).astype(np.int32)
        return discriminator, accumulated_pos
    


    #@vectorize(['int32(int32,int32,int32,int32)'], target = 'cuda')
    @staticmethod
    @njit(parallel = True)
    def test_with_bleaching(d,pos,x_test,y_test):
        right = 0
        wrong = 0
        images = x_test
        lable = y_test
        ct = 0.02
        b=1
        
        for i in range(len(images)):
            image = images[i]
            actual_lable = lable[i]
            
            total_sum=[]
            
            for ix in range(dis_number):
                
                t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
                sum_of_ram_output = 0
                dis = d[ix]
                
                for i in range(int(nodes)):
                    part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
                    ratina_for_one_ram = t_ratina[i]
                    
                    n = []                                                                
                    for pix in ratina_for_one_ram:
                        if image[(pix-1)]>=1:
                            n.append(1)
                        else:
                            n.append(0)
                    
                    num = 0
                    for i in range(no_of_rand_pix_selec):
                        num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
                    address_of_that_ram = (int)(num)
                
                    for key in range(len(part)):
                        prt = part[key]
                        if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                            sum_of_ram_output += 1
                
                total_sum.append(sum_of_ram_output)        
        
            max_sum = 0
            sec_max = 0
            idx = 0
            
            for i in range(len(total_sum)):
                if max_sum < total_sum[i]:
                    max_sum = total_sum[i]
                    idx = i
                    
            for j in range(len(total_sum)):
                if sec_max < total_sum[j] and j!=idx:
                    sec_max = total_sum[j]
                    
            index_of_dis = idx
            if index_of_dis == actual_lable:
                right += 1
            else:
                wrong += 1
            
            if max_sum == sec_max:
                confidence = 0
            else:
                confidence = 1 - float(sec_max)/float(max_sum)
                if confidence < ct:
                    b += 1
        
        return right,wrong

In [4]:
input_size = 28*28
no_of_rand_pix_selec = 2**(3)     ## ** (must) no_of_rand_pix_selec = 2^(n) where n is 0,1,2...
nodes = int(input_size/no_of_rand_pix_selec)    #98
ram_address_count = 2**(no_of_rand_pix_selec)   #256
dis_number = 10                #10 i.e number of lables

In [5]:
(tx_train, ty_train), (tx_test, ty_test) = tf.keras.datasets.mnist.load_data()
#print(tx_train.shape)
px_train, py_train, px_test, py_test = preprocessing(tx_train, ty_train, tx_test, ty_test)
print(px_train.shape)
print(px_train[80].shape)
#print(px_train[80])

(60000, 784)
(784,)


In [6]:
w = WiSARDrp(input_size,no_of_rand_pix_selec,nodes,ram_address_count,dis_number)
d, acc_pos = w.discriminator()
print(d.shape)

(10, 25088, 2)


In [8]:
starttrain = time.time()
#@njit
def ff():
    #index1 = np.zeros((2,),dtype=np.int32)
    w.train_with_bleeching(d,acc_pos,px_train[0:60000],py_train[0:60000])

ff()
endtrain = time.time()
print("time train = ",endtrain - starttrain)

AttributeError: 'WiSARDrp' object has no attribute 'train_with_bleeching'

In [ ]:
starttest = time.time()
right,wrong = w.test_with_bleaching(d,acc_pos,px_test[0:10000],py_test[0:10000])
endtest = time.time()
print("time test = ",endtest - starttest)

In [ ]:
print("number of right result = ",right)
print("number of wrong results = ",wrong)
accuracy = ((right)/(right+wrong))*100
print("accuracy by testing the model =",accuracy)
#    end = time.time()
#    print("total time = ",end - start)

In [ ]:
@njit(parallel = True)
def train_with_bleeching(d,pos,x_train, y_train,index1):
    B = 300    
    images = x_train
    lable = y_train    
    image = images
    num = lable
    all_ram_of_selected_discriminator = d[num]
    t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
    for i in range((int)(nodes)):
        part = all_ram_of_selected_discriminator[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
        ratina_for_one_ram = t_ratina[i]
                
            #threshold = 0               
        n = []                                                                
        for ix in range(len(ratina_for_one_ram)):
            pix = ratina_for_one_ram[ix]
            if image[(pix-1)]>=1:
                n.append(1)
                    #threshold = threshold + 1     
            else:
                n.append(0)
                
        num = 0
        for i in range(no_of_rand_pix_selec):
            num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
            
        address_of_that_ram = (int)(num)
        for key in range(ram_address_count):
            index = part[key]
            index1 = index
                #if threshold >= 1:          
            if index[0] == address_of_that_ram and index[1] < B:
                index1[1] += 1
            else:
                if index[1] != 0 and index[1] != 0:
                    index1[1] -= 1
    return index1

In [56]:
import time
@njit(parallel = True)
def test_with_bleaching(d,pos,x_test,y_test):
    right = 0
    wrong = 0
    images = x_test
    lable = y_test
    b=1
        
    #for i in range(len(images)):
    image = images
    actual_lable = lable
            
    ix = actual_lable       
    #for ix in range(dis_number):
                
    t_ratina = pos[int((nodes*ix)):(int)((nodes*ix+nodes))]
                
    sum_of_ram_output = 0
    dis = d[ix]
                
    for i in range(int(nodes)):
        part = dis[(ram_address_count*i):(ram_address_count*i+ram_address_count)]
        ratina_for_one_ram = t_ratina[i]
                    
        n = []                                                                
        for pix in ratina_for_one_ram:
            if image[(pix-1)]>=1:
                n.append(1)
            else:
                n.append(0)
                    
        num = 0
        for i in range(no_of_rand_pix_selec):
            num = (n[i])*(10**((no_of_rand_pix_selec-1)-i)) + num
                    
        address_of_that_ram = (int)(num)
                
        for key in range(len(part)):
            prt = part[key]
            if prt[0] == address_of_that_ram and prt[1]>=b:           #have to change this
                sum_of_ram_output += 1
            
            
    if sum_of_ram_output/nodes >= 0.75:
        right += 1
    else:
        wrong += 1
        
        
    return right,wrong                
                
                

@njit
def IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel):
    stabilityqueue2 = stabilityqueue1
    c = np.zeros((quesize,3),dtype=np.int32) 
    b = np.zeros((4,3),dtype=np.int32)
    for queueIndex in range(quesize):
        queuevalue = np.reshape(stabilityqueue2[i,j,queueIndex,:],(3,1))
        if queuevalue[0][0] == pixel[0][0] and  queuevalue[1][0] == pixel[1][0] and  queuevalue[2][0] == pixel[2][0] :
            b = np.roll(stabilityqueue2[i,j,:queueIndex+1,:],3) 
            c[:queueIndex+1] = b
            c[queueIndex+2:] = stabilityqueue2[i,j,queueIndex+2:,:]
            stabilityqueue2[i,j,:,:] = c
            return stabilityqueue2,True
    #print(stabilityqueue2.shape)
    return stabilityqueue2,False    
    
    
    
#@njit
@njit
def bewis(frame,w,ID,stabilityqueue,out,index,d,pos):
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            #print(i)
            r,g,b = frame[i,j,:]
            #create binary endoder for each pixel
            a = np.zeros((256,3))
            a[0:r,0] = 1  
            a[0:g,1] = 1
            a[0:b,2] = 1
            #check if pixel color is in front of stability queue
            #print(a)
            front = np.reshape(stabilityqueue[i,j,0,:],(3,1)) 
            pixel = np.reshape(frame[i,j,:],(3,1))
            pixel_no = i*10+j
            #print(111)
            if front[0][0] == pixel[0][0] and  front[1][0] == pixel[1][0] and  front[2][0] == pixel[2][0] :
                w[i,j] += ID[i,j]
                #print(w1[i,j])
                ID[i,j] += 1
                #print(w1[i,j])
                #continue
            #print(333)    
            #stabilityqueue1,found = IsFoundinStabilityQueue(stabilityqueue1,i,j,pixel)
            #print(222)
            if front[0][0] != pixel[0][0] or  front[1][0] != pixel[1][0] or  front[2][0] != pixel[2][0] :
                stabilityqueue,found = IsFoundinStabilityQueue(stabilityqueue,i,j,pixel)
                if found:
                    ID[i,j] = 1
                #print(1)
                else:
                    stabilityqueue[i,j,:,:] = np.roll(stabilityqueue[i,j,:,:],1)
                #print(stabilityqueue1[i,j,0,:].shape)
                    stabilityqueue[i,j,0,:]= np.reshape(pixel,(3,))
                    w[i,j] = 0
                #print(2)
                    ID[i,j] = 1
            
            #print(w1[i,j])       #here i am getting 0
            #print(2)
            if w[i,j] > k:
                #print(w1[i,j])
                #print(pixel_no)
                #print(1)
                pixel_train = preprocessing(a)
                #print(pixel_train.shape)
                #index1 = np.zeros((2,),dtype=np.int32)
                #index1 = train_with_bleeching(d,acc_pos,pixel_train,pixel_no,index1)
                #index = index1
                B = 50    
                images = pixel_train
                lable = pixel_no   
                image = images
                num = lable
                all_ram_of_selected_discriminator = d[num]
                t_ratina = pos[(int)(nodes*num):(int)(nodes*num+nodes)]
            
                for inode in range((int)(nodes)):
                    part = all_ram_of_selected_discriminator[(ram_address_count*inode):(ram_address_count*inode+ram_address_count)]
                    ratina_for_one_ram = t_ratina[inode]
                
            #threshold = 0               
                    n = []                                                                
                    for ix in range(len(ratina_for_one_ram)):
                        pix = ratina_for_one_ram[ix]
                        if image[(pix-1)]>=1:
                            n.append(1)
                    #threshold = threshold + 1     
                        else:
                            n.append(0)
                
                    num = 0
                    for ino in range(no_of_rand_pix_selec):
                        num = (n[ino])*(10**((no_of_rand_pix_selec-1)-ino)) + num
            
                    address_of_that_ram = (int)(num)
                    for key in range(ram_address_count):
                        #print(3)
                        index = part[key]
                        #print(4)
                #if threshold >= 1:
                        #index.setflags(write=1)
                        if index[0] == address_of_that_ram and index[1] < B:
                            index[1] += 1
                        else:
                            if index[1] != 0 and index[1] != 0:
                                index[1] -= 1
                #print(pixel.shape)
                #print(pixel.dtype)
                #print(i,j)
                #print(a)
                #print(a.shape)
            #print(w1[i,j])
            #reconstruct colors from neurons and write it on BG image
            #print(5)
            pixel_test = preprocessing(a)
            #print(pixel_no)
            right,wrong = test_with_bleaching(d,acc_pos,pixel_test,pixel_no)
            #print(right)
            if right:
                
                out[i,j] = 0.
                #print(out1)
            if wrong:
                #print(5)
                out[i,j] = 1.
                #print(0)




stabilityqueue = np.zeros((240,250,quesize,3),dtype=np.int32)   #240,256  #480,640 for bird video, 720,1280 for E_IP1231_Day_1
w = np.zeros((240,250),dtype=np.int32)
ID = np.zeros((240,250),dtype=np.int32)
cap = cv2.VideoCapture('day.avi')
out = np.ones((240,250),dtype=np.float64)
index = np.zeros((2,),dtype=np.int32)
n = 0
    #frame_number = 0
no_of_frames = 0
start = time.time()
while(True):
    #for file in glob.glob('/home/iss/project/BEWiS/dataset2014/dataset/baseline/highway/input/*jpg'):
    #frame = cv2.imread(file)
    ret, frame = cap.read()
        #Display the resulting frame
        #print(frame.shape)
        #if n <= 3:
            #ret, frame1 = cap.read()
            #cv2.imshow('frame1',frame1)
            #frame = frame
            #print(frame1.shape)
            #n += 1
    width = 250
    height = 240
    dim = (width, height)
 
        # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) 
        #print(frame.shape)
        #print(1)
    bewis(resized,w,ID,stabilityqueue,out,index,d,acc_pos)
    no_of_frames += 1
    #print(w)
        #print(out)
    cv2.imshow('frame',resized)
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(out,kernel,iterations = 1)
    dilation = cv2.dilate(erosion,kernel,iterations = 1)
    cv2.imshow('out',dilation)
        #frame_number += 1
        #print(frame_number)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # When everything done, release the capture

end = time.time()
time = end - start
    
fps = no_of_frames/time
print(fps)
cap.release()
cv2.destroyAllWindows()      

0.1838625002146275
